In [1]:
import kfp

from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

In [94]:
authservice_session='authservice_session=MTYxNDI0NzA4N3xOd3dBTkV0SVVVVkNWVTlPUkZWS04waEVXazVOVjB0SldWVTNXRVpXVGt4VVR6UTBUelJYVlZSVFZFVkhUMGxSV0VKWlRVUXpUVUU9fMZX0CmJhl9c5iSDbK-mRTLJyks0YRYWlMMg6bbde9pc'
client = kfp.Client(host='http://ad7fc227388c3431f9c778285d481334-127942271.us-west-2.elb.amazonaws.com/pipeline', cookies=authservice_session)
client.list_experiments(namespace="admin")

{'experiments': [{'created_at': datetime.datetime(2021, 2, 22, 10, 16, 13, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'bc05a15c-5ac6-4cdf-935b-7a6451089d2c',
                  'name': 'sample',
                  'resource_references': [{'key': {'id': 'admin',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [3]:
model="resnet"

In [4]:
!cat model_archive_step/component.yaml

name: Model Archiver
inputs:
- {name: properties, type: String}
- {name: extraFiles, type: String, default: ""}
- {name: modelFile, type: String, default: ""}
- {name: serializedFile, type: String}
- {name: handlerFile, type: String, default: ""}
- {name: requirementsFile, type: String, default: ""}
outputs:
  - {name: output_directory, type: String}
implementation:
  container:
    image: jagadeeshj/model_archive_step:25-02-2021-13-51-56.943180
    command: ["/usr/local/bin/dockerd-entrypoint.sh",
        "--output_path", {outputPath: output_directory},
        "--properties", {inputValue: properties},
        "--serializedfile", {inputValue: serializedFile},
        "--handlerfile", {inputValue: handlerFile},
        "--modelfile", {inputValue: modelFile},
        "--extrafiles", {inputValue: extraFiles},
        "--requirements", {inputValue: requirementsFile}
    ]

In [139]:
# load components (note the components are not platform specific, but the importers are)
data_prep_op = load_component_from_file(f"data_prep_step/{model}/component.yaml")
train_model_op = load_component_from_file(f"training_step/{model}/component.yaml")
list_item_op = load_component_from_file("file/component.yaml")
download_op = load_component_from_file("web/component.yaml")
model_archive_op = load_component_from_file("model_archive_step/component.yaml")

In [142]:
@dsl.pipeline(
    name = "pytorchcnn"
)
def train_imagenet_cnn_pytorch(
    ):
        
    # data_prep_task = data_prep_op(input_data = "")
    # data_prep_task = data_prep_op(input_data = "gs://cloud-ml-nas-public/classification/imagenet/train*", vocab_file = "bert_base_uncased_vocab.txt", vocab_file_url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt")

    # temp_input = "gs://managed-pipeline-test-bugbash/20210130/pipeline_root/pavel/c14ec128-18d4-4980-b9f3-e1c6f4babb51/pytorchcnn-dj5sg-2878573190/output_data/prefix"
    #data_prep_task.outputs["output_data"])

    # train_model_task = (train_model_op(trainingdata = "").
    #     set_cpu_limit('4').
    #     set_memory_limit('14Gi')
    # )    
    # archive_prep_task = archive_prep_op(folder = train_model_task.outputs["modelcheckpoint"], model_name = "resnet")

    # train_model_task = (train_model_op(trainingdata = data_prep_task.outputs["output_data"], maxepochs = 2, 
    #     numsamples = 150, 
    #     batchsize = 16,
    #     numworkers = 2,
    #     learningrate = 0.001,
    #     accelerator = "")
    #     .set_cpu_limit('4').
    #     set_memory_limit('14Gi')
    # )

#     prop_task = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/properties.json", input = "")
#     model_task = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/model.py", input = prop_task.outputs['output'])
#     serial_task = download_op(url = "https://download.pytorch.org/models/resnet18-5c106cde.pth", input = model_task.outputs['output'])
#     extra_task = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/index_to_name.json", input = serial_task.outputs['output'])

#     list_item_task = list_item_op(directory = extra_task.outputs["output"])

#   Using paths 
#     model_archive_task = model_archive_op(
#         input_directory=extra_task.outputs['output'],
#         handlerfile="image_classifier")

#     Using files
    model_archive_task = model_archive_op(
        properties="https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/properties.json",
        modelfile="https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/model.py", 
        serializedfile="https://download.pytorch.org/models/resnet18-5c106cde.pth", 
        extrafiles="https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/index_to_name.json", 
        handlerfile="image_classifier")


    list_item_task = list_item_op(directory = model_archive_task.outputs["output_directory"])

#     deploy_model_task = deploy_model_op(
#         action = 'create',
#         model_name='pytorch',
#         default_model_uri='gs://kfserving-samples/models/pytorch/cifar10/',
#         namespace='admin',
#         framework='pytorch',
#         default_custom_model_spec='{}',
#         canary_custom_model_spec='{}',
#         autoscaling_target='0',
#         kfserving_endpoint=''
#     )


In [143]:
compiler.Compiler().compile(train_imagenet_cnn_pytorch, 'pytorch.tar.gz', type_check=True)
# Execute pipeline
run = client.run_pipeline("bc05a15c-5ac6-4cdf-935b-7a6451089d2c", 'pytorch-model', 'pytorch.tar.gz')